# PHYS512 Final Project: Stacking Analysis over Simulation of the $[CII]$ Signal

Prof: Jon Sievers, TAs: Marcus Merryfield, Rigel Zifkin, Daniel Coelho

by Patrick Horlaville

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy.units as u # I use astropy units in everything, very helpful for bookkeeping/debugging

from lim import lim # The base intensity mapping object

matplotlib.rcParams.update({'font.size': 18,'figure.figsize':[8,7]}) # Make figures better
#matplotlib.rcParams,update({}'figure.figsize'] = [8,7]

## Introduction

Galaxy formation and star forming processes at high redshift, crucial to the history of our modern Universe, are becoming the focus point of many astrophysical observational endeavours in the advent of the necessary observing tools to start probing distant galaxies. One of the target frequencies of this investigation is the 158 $\mu$m rest-frame $\rm{[CII]}$ line. As a major coolant of the neutral atomic gas of the interstellar medium, the $\rm{[CII]}$ signal is proportional to the heating rate and the fraction of atomic gas. Additionally, it is the strongest emission line of star forming galaxies at rest-frame far-infrared wavelength, which makes it the ideal tracer for galactic star formation rate (SFR) [1, 2].


Observations of local galaxies have revealed a linear scaling relationship the $\rm{[CII]}$ luminosity $L_{\rm{CII}}$ and SFR [e.g., 3]. At higher redshifts ($z > 5$), various small scale surveys have yielded divergent results: some report that the linear scaling from the lower redshifts persists [e.g., 4], while others report a systematic decrease of the $\rm{[CII]}$ luminosity, the so-called "$\rm{CII}$ deficit" [e.g., 5]. Hence, it is important in order to shed light on this problem to try to build better models of the $\rm{[CII]}$ signal.

## Previous Work [from this Summer]

### Context

This project is led following my research project at the Canadian Institute for Theoretical Astrophysics (CITA) during Summer 2022, during which I implemented a new model for the $\rm{[CII]}$ signal within Prof. Dick Bond's group. The first step of the project was to revive an old line intensity mapping package called `limlam_mocker` developed in 2018 by Dr. George Stein and Dr. Dongwoo Chung, my supervisor at CITA. The final goal was to add a new $\rm{[CII]}$ model to this package, which would allow to retrieve, for a given lightcone of simulated halos (around $z \sim 5 - 8$), the $\rm{[CII]}$ signal of those halos from their masses (in our lightcone, halo masses are of order $10^{10} \sim 10^{12} M_{\odot}$).

### The Model

The model arose from a newly-derived relationship for $L_{\rm{CII}}$ found by another member of prof. Dick Bond's group, Dr. Lichen Liang. 

By using FIRE (Feedbacks In Realistic Environments) full cosmological volume simulations between $z \sim 0 - 8$, Liang et al. (in prep.) observed the following relationship of galactic halo properties:

[1] : $\frac{L_{\rm{[CII]}}/L_{\odot}}{\rm{SFR}/(M_{\odot}yr^{-1})} \propto f_{\rm{[CII]}}\overline{Z}_{\rm{gas}}t_{\rm{dep}}\overline{n}_{\rm{gas}}$

where:

- $L_{\rm{[CII]}} = \rm{[CII]}$luminosity

- $f_{\rm{[CII]}}$ = fraction of total gas mass that comes from HI and HII regions

- $\overline{Z}_{\rm{gas}}$ = gas metallicity

- $t_{\rm{dep}} \equiv \frac{M_{\rm{gas}}}{\rm{SFR}}$ = galactic gas depletion time scale

- $\overline{n}_{\rm{gas}}$ = gas density statistical average

By multipliying both sides of the equation by the SFR, and if we suppose that the galactic $\rm{HI}$ mass is the main $\rm{[CII]}$ contributor across each galaxy (such that $M_{\rm{HI}} \sim f_{\rm{[CII]}} M_{\rm{gas}}$), we retrieve a proportionality relationship:

[2] : $L_{\rm{[CII]}} \propto M_{\rm{HI}}$

Which we parametrize as:

[3] : $L_{\rm{[CII]}}/L_{\odot} = \alpha_{\rm{[CII]}} \times M_{\rm{HI}}/M_{\odot}$

Where $\alpha_{\rm{[CII]}}$ is the proportionality coefficient the $\rm{HI}$ mass and the $\rm{[CII]}$ luminosity. We would like express $M_{\rm{HI}}$ as a function of halo mass $M_{\rm{halo}}$ and we would like to determine an estimate for $\alpha_{\rm{[CII]}}$ at our redshift ($z \sim 5 - 8$). The literature provides prescriptions for $M_{\rm{HI}}(M_{\rm{halo}} relationships, e.g. Villaescua-Navarro et al. (2018):

[4] : $M_{\rm{HI}}(M_{\rm{halo}}) = M_0 (\frac{M_{\rm{halo}}}{M_{\rm{min}}})^{\alpha} {exp}(- (\frac{M_{\rm{min}}}{M_{\rm{halo}}})^{0.35})$

Where $M_0$, $M_{\rm{min}}$ and $\alpha$ are the fit parameters to the relationship. Here we use their parametrization for $z = 5$: $M_0$ = $1.9 \times 10^9 M_{\odot}$, $M_{\rm{min}}$ = $2.0 \times 10^{10} M_{\odot}$ and $\alpha$ = 0.74.

We use the SFR as a mediator to find our parameter $\alpha_{\rm{[CII]}}$: for a particular value of SFR, we would like to know what are the corresponding halo mass $M_{\rm{halo}}$ and the $\rm{[CII]}$ luminosity of a halo in our redshift range. For the $M_{\rm{halo}}/\rm{SFR}$ relationship, we can use a standard prescription, such as from Behroozi et al., 2013. This SFR model is incorporated into the `limlam_mocker` package, so we can interpolate the $M_{\rm{halo}}$ from the SFR handily. For the $L_{\rm{[CII]}}/\rm{SFR}$ relationship, we can use another result from Liang et al. (in prep.):

[5] : $\frac{L_{\rm{[CII]}}}{L_{\odot}} \sim 10^7 \times \frac{\rm{SFR}}{M_{\odot} yr^{-1}}$

So we can just pick a value of SFR, $\rm{SFR}^*$, and find the corresponding $L_{\rm{[CII]}}^*$ and $M_{\rm{halo}}^*$ with the models described above. With $M_{\rm{halo}}^*$, we can find $M_{\rm{HI}}^*$ with the Villaescua-Navarro et al. (2018) model.

By using $\rm{SFR}^* = 1 \frac{M_{\odot}}{yr}$, we retrieved a value of $\alpha_{\rm{[CII]}} \sim 0.005$ for our redshift range. By substituting this coefficient in equation [3] and by using equation [4] to parametrize $M_{\rm{HI}}$, we have a recipe to retrieve halos' $\rm{[CII]}$ luminosity $L_{\rm{[CII}}$ from their mass $M_{\rm{halo}}$.

### The Forecast

Now that we are able to model the $\rm{[CII]}$ of halos from their masses, we would like to produce forecasts of future experiments that aim towards observing the $\rm{[CII]}$ signal of halos in our redshift range ($z \sim 5 - 8$). The CCAT-prime collaboration and their upcoming Fred Young Submillimeter Telescope (first light expected in 2024) have as a primary science objective to probe the $\rm{[CII]}$ line in our $z$ range, hence it would be relevant to model what would their observations look like given our model of the signal.